In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


c:\Users\Felip\Miniconda3\envs\MLearn\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class SentientAnalizer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, vocab_size, embedding_dim):
        super(SentientAnalizer, self).__init__()
        """
        [Vocab_size -> Embedding_size]
        input_size: [Seq, Batch, Inputs]
        """
        # input_size == 1
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.Vocab_Embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.LSTM = nn.LSTM(self.embedding_dim, self.hidden_size,num_layers=2, batch_first=True)
        self.Linear1 = nn.Linear(self.hidden_size, self.hidden_size)
        self.Linear2 = nn.Linear(self.hidden_size, self.output_size)


    def forward(self,x, hidden):
        emb = self.Vocab_Embedding(x).view(x.shape[0],x.shape[1],-1)
        out, hidden = self.LSTM(emb, hidden)
        out = self.Linear1(out)
        out = self.Linear2(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(2, batch_size, self.hidden_size),
                torch.zeros(2, batch_size, self.hidden_size))

In [51]:
class StockPredictor(nn.Module):

    def __init__(self, input_size_stock,input_size_sentiment, hidden_size, num_layers, max_length):
        super(StockPredictor, self).__init__()
        """
        [Vocab_size -> Embedding_size]
        input_size: [Seq, Batch, Inputs]
        """
        # input_size == 1
        self.input_size_stock = input_size_stock
        self.input_size_sentiment = input_size_sentiment
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.attn_hidd = nn.Linear(self.hidden_size, max_length)
        self.attn_out = nn.Linear(self.hidden_size*2, self.hidden_size)
        self.CombinedLayerSS = nn.Linear(self.input_size_stock+self.input_size_sentiment, self.hidden_size)
        self.LSTM = nn.LSTM(self.input_size_sentiment+self.input_size_stock, self.hidden_size,num_layers=num_layers, batch_first=True)
        self.Linear1 = nn.Linear(self.hidden_size, self.hidden_size)
        self.Linear2 = nn.Linear(self.hidden_size, self.input_size_stock)


    def forward(self,x, hidden, stock_news):
        attn_wights = F.softmax(self.attn_hidd(hidden[0][0]), dim=1)
        context = torch.bmm(attn_wights.unsqueeze(0), stock_news.unsqueeze(0))
        x_SS = torch.cat((x,context),dim=2)
        out, hidden = self.LSTM(x_SS, hidden)
        out = self.Linear1(out)
        out = self.Linear2(out)
        return out, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))

In [55]:
Senti = SentientAnalizer(1, 128, 256, 10000, 128)
Stock = StockPredictor(5,256,256,5,10)
# input of sentient
batch_size_senti = 10
# input of Sentiment
# InputSize [Batch, Seq, Input] = [Cantidad de noticias, Cantidad de palabras, Cantidad de palabras]
#                                                                               <Recuerda que es 1>
seq_len = 5 # padd news to same length
noticias = torch.zeros(batch_size_senti,seq_len,1, dtype=torch.long) # example vector
hidde_senti = Senti.init_hidden(batch_size_senti)
out_senti, hidden_senti = Senti(noticias, hidde_senti)
print (out_senti.shape) # [Batch, Seq, Output] -> [10, 5, 256] 
# Pero solo queremos las ultimas salidas, osea [:,-1,:]
out_senti = out_senti[:,-1,:]
print (out_senti.shape) # [Batch, Output] -> [10, 256]
# input of Stock
batch_size_stock = 1
# InputSize [Batch, Seq, Input] = [Cantidad de noticias, Cantidad de palabras, Cantidad de variables]
contexto_len = 1
vars_stock = 6
x = torch.zeros(batch_size_stock,contexto_len,vars_stock, dtype=torch.float)
hidde_stock = Stock.init_hidden(batch_size_stock)
#print (x.shape, hidde_stock[0].shape, out_senti.shape)
out_stock, hidden_stock = Stock(x, hidde_stock, out_senti)
print (out_stock.shape) # [Batch, Seq, Output] -> [1, 1, 5]

torch.Size([10, 5, 256])
torch.Size([10, 10, 1, 256])


RuntimeError: batch2 must be a 3D tensor

In [75]:
# count the number of parameters
num = 0
for p in Senti.parameters():
    num += p.numel()
print (num)

1593728


In [ ]:
class Dataset (torch.utils.data.Dataset):
    def __init__(self, data_stock, news, target):
        self.data = data
        self.labels = labels
        self.news = news
        self.news_labels = news_labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index], self.news[index], self.news_labels[index]

In [ ]:
# predict next 5 days
for i in range(5):
    out_stock, hidden_stock = Stock(out_stock, hidden_stock, out_senti)
    print (out_stock.shape)